# clean

> This module contains all the various cleaning options supported.

In [ ]:
#| default_exp clean

In [ ]:
#| export
import re

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def normalize_whitespace(
    text: str, # The text to normalize
) -> str: # The normalized text
    """
    Replace runs of whitespace with a single space, strip leading/trailing whitespace,
    and normalize all newlines to unix-style newlines.
    """
    return re.sub(r"\s+", " ", text.strip()).replace("\r\n", "\n")

In [ ]:
#| export
def remove_empty_lines(
    text: str, # The text to remove empty lines from
) -> str: # The text with empty lines removed
    """
    Remove empty lines from the text.
    """
    return re.sub(r"^\s*$", "", text, flags=re.MULTILINE)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()